In [2]:
import numpy as np
import scipy
import scipy.stats as sps
import scipy.fftpack as spfft
import scipy.signal as spsig
import matplotlib.pyplot as plt
from astropy.timeseries import LombScargle
import scipy.integrate as integrate
from scipy.integrate import quad
from scipy.signal import savgol_filter
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
import stingray

In [5]:
def simlc(dt, ntimes, expquery, rms, psmodel, pspar):
    '''
    dt       :  bin size
    ntimes   :  number of time bins( power of 2)
    expquery :  selection between non linear variability and non skewed flux distribution
    rms      :  rms value of the power spectrum in the given frequency range
    psmodel  :  power spectrum model
    pspar    :  power spectrum model paramters
    
    '''
    # Generates simulated light curve with ntimes consecutive time bins of bin size dt, required output rms
    # (set to zero to use the power spectrum normalisations as given) and a given power-spectral model and input
    # parameters
    f_min = 1./(dt*ntimes) # Define minimum and maximum sampled frequency
    f_max = 1/(2.*dt)
    params = tuple(pspar) # Need to do this since the quad function requires function arguments as a tuple
    int_rmssq, err = quad(psmodel, f_min, f_max, args=params)
    #print("Integrated rms = ",np.sqrt(int_rmssq))
    if (rms > 0.): # If rms > 0 then correct the power spectrum normalisation to obtain output light curve with the 
        # input rms value (otherwise the output is based on the normalisation given in pspar)
        pspar[0] = pspar[0]*(rms**2)/int_rmssq
        int_rmssq = rms**2
        params = tuple(pspar)
        #print("Correcting rms to: ",np.sqrt(quad(psmodel, f_min, f_max, args=params)[0]))    
                                            
    if (expquery == 'y'): # If we want to exponentiate the light curve to ensure fluxes are positive and 
        # flux distribution is lognormal, similar to real data.  The input normalisation is corrected so the output
        # has the same fractional rms as the input rms (assumed to be fractional)
        exp_pspar = np.copy(pspar)
        rmssq = int_rmssq
        linrmssq = np.log(rmssq+1.0)
        exp_pspar[0] = exp_pspar[0]*(linrmssq/rmssq)
        lc = tksim(dt, ntimes, psmodel, exp_pspar)
        lc = np.exp(lc)
    else:
        lc = 1. + tksim(dt, ntimes, psmodel, pspar) # This is just a linear model with normally distributed fluxes,
        # if the rms is large there is a risk that some fluxes go negative.

    lc = lc/np.mean(lc)  # Output is normalised to mean of 1
        
    #print("Output light curve standard deviation = ",np.std(lc))
    return lc

def tksim(dt, ntimes, psmodel, pspar):
    '''
    dt       :  bin size
    ntimes   :  number of time bins( power of 2)
    psmodel  :  power spectrum model
    pspar    :  power spectrum model paramters
    
    '''
# Implementation of Timmer & Koenig method to simulate a noise process for an arbitrary power-spectral shape 
# (Timmer & Koenig, 1995, A&A, 300, 707), based on an original Python implementation by Dimitrios Emmanoulopoulos
    nfreq = ntimes//2
    f = np.arange(1.0, nfreq+1, 1.0)/(ntimes*dt)
    modpow = psmodel(f, *pspar) * nfreq/dt

    ft_re=np.random.normal(scale = np.sqrt(modpow*0.5))
    ft_im=np.random.normal(scale = np.sqrt(modpow*0.5))
    
    ft_pos = ft_re + ft_im*1j
    ft_neg = np.conj(ft_pos) # sets negative frequencies as complex conjugate of positive (for real-valued LC)
    ft_full=np.append(ft_pos,ft_neg[nfreq-2::-1]) # append -ve frequencies to +ve.  Note that scipy.fftpack orders
    # the FT array as follows: y[0] = zero freq, y[1:nfreq-1] ascending +ve freq values, y[nfreq:2*nfreq-1] ascending
    # (i.e. less -ve) -ve freq values.  This means that the Nyquist freq value used at y[nfreq] is actually the -ve
    # frequency value.
    # For our even-valued nfreq this doesn't matter since we must set the Nyquist freq value to be real anyway 
    # (see below).  
    ft_full=np.insert(ft_full,0,complex(0.0,0.0))  # Set zero-freq (i.e. 'DC component' or mean) to zero
    ft_full[nfreq]=complex(ft_full.real[nfreq],0.0) # For symmetry need to make Nyquist freq value real - note that
    # neglecting to do this causes a small imaginary component to appear in the inverse FFT
    ift_full=np.fft.ifft(ft_full)                            
    lc=ift_full.real
    return lc                                            


def lorentz_q(f, lor_rmssq, f_pk, q):  
# Form of the Lorentzian function defined in terms of peak frequency and quality factor q
# e.g. see Pottschmidt et al. 2003, A&A, 407, 1039 for more info.
# This form is commonly used because f_pk corresponds to the peak that is seen when plotting
# the Lorentzian using frequency*power, so is more intuitive than using the centroid for the characteristic
# frequency.
    f_res=f_pk/np.sqrt(1.0+(1.0/(4.0*q**2)))
    r=np.sqrt(lor_rmssq)/np.sqrt(0.5-np.arctan(-2.0*q)/np.pi)
    powmod = ((1/np.pi)*2*r**2*q*f_res)/(f_res**2+(4*q**2*np.square(f-f_res)))
    return powmod

def lorentz_fwhm(f, lor_rmssq, f_cent, fwhm): 
# Traditional form of the Lorentzian function, with a centroid and FHWM
    powmod = lor_rmssq * (fwhm/(2.*np.pi))/((f-f_cent)**2 + (fwhm/2.)**2)
    return powmod

def bend_pl(f, norm, f_bend, alph_lo, alph_hi, sharpness):
    # Bending power-law with two slopes, modified from Summons et al. 2007 
    # (http://adsabs.harvard.edu/abs/2007MNRAS.378..649S)
    # to include 'sharpness' parameter.  Sharpness = 1 same as simpler Summons et al. model, larger values
    # give a sharper transition from one slope to the other.
    # Typical slopes for AGN would be alph_lo=-1, alph_hi=-2 to -3
    powmod = (norm*(f/f_bend)**alph_lo)/(1.+(f/f_bend)**(sharpness*(alph_lo-alph_hi)))**(1./sharpness)
    return powmod

def dbl_bend_pl(f, norm, f_bend_lo, f_bend_hi, alph_lo, alph_med, alph_hi, sharpness):
    # As bend_pl but now with two bends.  If AGN look like BHXRBs the low-frequency slope would be ~0,
    # with medium and high slopes like that for the single-bend case.  This shape could be mimicked by 
    # multiple Lorentzians
    powmod = (norm*(f/f_bend_lo)**alph_lo)/(1.+(f/f_bend_lo)**((sharpness*(alph_lo-alph_med)))*
        (1.+(f/f_bend_hi)**(sharpness*(alph_med-alph_hi))))**(1./sharpness)
    return powmod

def gaussian(f, f_peak, width, amplitude ):
    
    return amplitude * np.exp(-(f-f_peak)**2/2*width**2)
    
    
def bend_pl_lorentz(f, norm, f_bend, alph_lo, alph_hi, sharpness, lor_rmssq, f_pk, q):
    f_res=f_pk/np.sqrt(1.0+(1.0/(4.0*q**2)))
    r=np.sqrt(lor_rmssq)/np.sqrt(0.5-np.arctan(-2.0*q)/np.pi)
    powmod_lorentz = ((1/np.pi)*2*r**2*q*f_res)/(f_res**2+(4*q**2*np.square(f-f_res)))
    
    powmod_bend_pl = (norm*(f/f_bend)**alph_lo)/(1.+(f/f_bend)**(sharpness*(alph_lo-alph_hi)))**(1./sharpness)
    
    return powmod_bend_pl+powmod_lorentz

In [6]:
def plot_simulate(psmodel, pspar, dt=10.0, ntimes=2**10, expquery = 'n', rms=0.6):
    params = tuple(pspar)

    lc = simlc(dt, ntimes, expquery, rms, psmodel, pspar)

    time = np.linspace(start=dt,stop=(dt*ntimes),num=ntimes)

    fig,ax = plt.subplots(1,2, figsize=[17,6])
    ax[0].plot(time,lc, 'k-', lw=0.5)
    ax[0].set_xlabel('Time points', fontsize=16)
    ax[0].set_ylabel('Normalised flux', fontsize=16)
    ax[0].set_title('Light curve', fontsize=19)
    # plot the model

    minfreq = 1./(ntimes*dt)
    freq = np.linspace(start=minfreq,stop=(minfreq*ntimes/2),num=ntimes//2)
    ax[1].plot(freq,np.multiply(psmodel(freq,*pspar),(ntimes//2)/dt), 'r-', lw=3)
    ax[1].set_xlabel(r'$\nu$',fontsize=16)
    ax[1].set_ylabel(r'$\nu P_{\nu}$',fontsize=16)
    ax[1].set_yscale('log')
    ax[1].set_xscale('log')
    ax[1].set_title('Power spectrum', fontsize=19)
    plt.tight_layout()
    plt.show()
    
    return time, lc

def moving_average_interpolated(data, window_size):
    
    if window_size <= 0:
        raise ValueError("Window size must be greater than 0")
    if len(data) < window_size:
        raise ValueError("Window size is larger than the length of the data")

    moving_averages = []

    for i in range(len(data) - window_size + 1):
        window = data[i:i + window_size]
        average = sum(window) / window_size
        moving_averages.append(average)

    # Interpolate to match the input data size
    interpolated_moving_averages = np.interp(
        range(len(data)),
        range(window_size - 1, len(data)),
        moving_averages
    )

    return interpolated_moving_averages

In [7]:
from ipywidgets import interact, widgets
from IPython.display import display, clear_output

In [6]:
(2**13*1)/365

22.443835616438356

In [8]:

psmodel = bend_pl
pspar = [100.,1e-3,-1,-1,5.0] 

#psmodel = lorentz_q
#pspar = [10, 1e-2, 0.5]

#psmodel = bend_pl_lorentz
#pspar = [0,1e-3,-0,-0,10.0, 100, 1e-2, 10 ]

bend_freq_array = np.linspace(1/(10*365), 1/(0.1*365), 100)
lower_slope_array = np.linspace(-2, 0, 50)
sharpness_array = np.linspace(1, 15, 50)
dt=1
ntimes=2**13
expquery = 'n'
rms=0.15

#for i,val in enumerate(bend_freq_array):
#for i,val in enumerate(sharpness_array):
for i,val in enumerate(lower_slope_array):    
    pspar[2] = val
    params = tuple(pspar)
    np.random.seed(10)
    lc = simlc(dt, ntimes, expquery, rms, psmodel, pspar)

    time = np.linspace(start=dt,stop=(dt*ntimes),num=ntimes)

    fig,ax = plt.subplots(1,2, figsize=[17,6])
    ax[0].plot(time/365,lc, 'k-', lw=0.5)
    ax[0].set_xlabel('Time points', fontsize=16)
    ax[0].set_ylabel('Normalised flux', fontsize=16)
    ax[0].set_title('Light curve', fontsize=19)
    #ax[0].set_ylim(0.2, 2)
    
    # plot the model

    minfreq = 1./(ntimes*dt)
    freq = np.linspace(start=minfreq,stop=(minfreq*ntimes/2),num=ntimes//2)
    ax[1].plot(freq,np.multiply(psmodel(freq,*pspar),(ntimes//2)/dt), 'r-', lw=3)
    ax[1].set_xlabel(r'$\nu$',fontsize=16)
    ax[1].set_ylabel(r'$\nu P_{\nu}$',fontsize=16)
    ax[1].set_yscale('log')
    ax[1].set_xscale('log')
    ax[1].set_ylim(10**-4, 10**8)
    ax[1].set_title('Power spectrum', fontsize=19)
    plt.tight_layout()
    plt.suptitle(str(i)+'  '+ str(np.round(val, decimals=5)) + '  '+str(np.round(1/val/365, decimals=3)))
    display(fig)
    #plt.pause(0.5)
    clear_output(wait=True)
    plt.clf()
    #plt.close()
#time,lc = plot_simulate(psmodel, pspar)

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>

...

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>

<Figure size 1700x600 with 0 Axes>